In [3]:
from datetime import datetime 

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [4]:
from __future__ import division, print_function, absolute_import
import tensorflow as tf 

# Import MNIST data 
from tensorflow.examples.tutorials.mnist import input_data 
mnist = input_data.read_data_sets("/tmp/data", one_hot = True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [231]:
tf.reset_default_graph()

In [232]:

# Training Parameters 
learning_rate = 0.001
beta = 0.1
num_steps = 500
batch_size = 128
display_step = 10
thres = tf.math.sqrt(2*0.0005)

# Network Parameters 
num_input = 784 #MNIST data input 
num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units 

# tf Graph input 
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [233]:
# Create some wrappers for simplicity 
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation 
    x = tf.nn.conv2d(x, W, strides = [1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    # MaxPool2D wrapper 
    return tf.nn.max_pool(x, ksize=[1,k,k,1], strides=[1,k,k,1],
                         padding = 'SAME')

# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features 
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    # Convolution Layer 
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)
    
    # Convolution Layer 
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)
    
    # Fully Connected Layer 
    # Reshape conv2 output to fit fully connected layer input 
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout 
    fc1 = tf.nn.dropout(fc1, dropout)
    
    # Output, class predictin 
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    
    return out 

In [234]:
def tf_hard_threshold(x):
    condition = tf.less(tf.abs(x), thres)
    u = tf.where(condition, tf.zeros_like(x), x)
    return u

In [235]:
# Store layers weight & bias 
weights = {
    # 5x5 conv, 1 input, 32 outputs 
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32]), name = "wc1"),
    # 5 x5 conv, 32 inputs, 64 outputs 
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64]), name = "wc2"),
    # fully connected, 7*7*64 inputs, 1024 outputs 
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024]), name = "wd1"),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]), name = "wout")
}
biases = {
    'bc1': tf.Variable(tf.random_normal([32]), name = "bc1"),
    'bc2': tf.Variable(tf.random_normal([64]), name = "bc2"),
    'bd1': tf.Variable(tf.random_normal([1024]), name = "bd1"),
    'out': tf.Variable(tf.random_normal([num_classes]), name = "bout")
}

bd = tf.print(biases['bd1'])

# Construct model 
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer 
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits = logits, labels= Y))

tvars = tf.trainable_variables()
#tuars = tvars
tuars2 = tf_hard_threshold(tvars[2])


grad = tf.gradients(loss_op, tvars)
new_grad = grad[2] + beta*tf.subtract(tvars[2], tuars2)


In [260]:
grad_2 = []
for i in range(2):
    grad_2.append(grad[i])
grad_2.append(new_grad)
for i in range(3,len(grad)):
    grad_2.append(grad[i])

In [261]:
grad_2

[<tf.Tensor 'gradients/Conv2D_grad/Conv2DBackpropFilter:0' shape=(5, 5, 1, 32) dtype=float32>,
 <tf.Tensor 'gradients/Conv2D_1_grad/Conv2DBackpropFilter:0' shape=(5, 5, 32, 64) dtype=float32>,
 <tf.Tensor 'add_3:0' shape=(3136, 1024) dtype=float32>,
 <tf.Tensor 'gradients/MatMul_1_grad/MatMul_1:0' shape=(1024, 10) dtype=float32>,
 <tf.Tensor 'gradients/BiasAdd_grad/BiasAddGrad:0' shape=(32,) dtype=float32>,
 <tf.Tensor 'gradients/BiasAdd_1_grad/BiasAddGrad:0' shape=(64,) dtype=float32>,
 <tf.Tensor 'gradients/Add_grad/Reshape_1:0' shape=(1024,) dtype=float32>,
 <tf.Tensor 'gradients/Add_1_grad/Reshape_1:0' shape=(10,) dtype=float32>]

In [262]:

optimizer = tf.train.AdamOptimizer(learning_rate= learning_rate)


train_op = optimizer.apply_gradients(zip(grad_2, tvars))

# Evaluate mdel 
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [263]:
writer = tf.summary.FileWriter("/home/sophia/Documents/math296/tmp/histogram_example")

summaries = tf.summary.merge_all()

In [264]:
# Start training 
with tf.Session() as sess:
    
    
    # Run the initializer 
    sess.run(init)
    
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: dropout})
        if step% display_step == 0 or step == 1:
            # Calculate batch loss and accuracy 
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X:batch_x,
                                                                 Y:batch_y,
                                                                keep_prob: 1.0})
            print("Step "+ str(step) + ", Minibatch Loss= " + \
                 "{:.4f}".format(loss) + ", Training Accuracy=" + \
                 "{:.3f}".format(acc))
    variable_dict = {}
    for var in tf.trainable_variables():
        print(var.name)
        print(var.eval())
        variable_dict[var.name] = var.eval()

    print("Optimization Finished!")
    
    # Calculate accuracy for 265 MNIST test images 
    print("Testing Accuracy: ", \
         sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                       Y: mnist.test.labels[:256],
                                       keep_prob: 1.0}))
        

TypeError: 'Operation' object is not callable

In [266]:
for key,val in variable_dict.items():
    print (key)